<a href="https://colab.research.google.com/github/pmontman/TSForecasting/blob/master/experiments/forecastingdata_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for accessing Forecastingdata feature extraction tools using python

The feature calculation and other utilities are implemented in R, but we can access them *transparently* from python using the awesome rpy2 package.
This notebook show an example for it.


It has two parts. Part 1 is the preparation of the environment so everything
rund directly on python. Part2 is an example of how to use the tools directly from python.

## Part 1) Preparation of the environment

We load the R extension for jupyter, which interacts nicely with python

In [ ]:
#@title
%load_ext rpy2.ipython

Clone the repository, for reproducibility, no need to clone everytime.

In [ ]:
!git clone https://github.com/rakshitha123/TSForecasting

fatal: destination path 'TSForecasting' already exists and is not an empty directory.


We now prepare the R environment for processing: installing and loading the required packages

In [ ]:
%%R
devtools::install_github("hendersontrent/catch22")
install.packages("tsibble")
install.packages("forecast")
install.packages("tsfeatures")
library("Rcatch22")

R[write to console]: Skipping install of 'Rcatch22' from a github remote, the SHA1 (a93d1a9b) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/tsfeatures_1.0.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 146035 bytes (142 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to co

In [ ]:
#%%R
#print(length(SEASONALITY_VALS))
#print(FREQUENCIES)

In [ ]:
%%R
library(tidyverse)
library(Rcatch22)

BASE_DIR <- "TSForecasting"

source(file.path(BASE_DIR, "utils", "data_loader.R", fsep = "/"))

# The name of the column containing time series values after loading data from the .tsf file into a tsibble
VALUE_COL_NAME <- "series_value"

# seasonality values corresponding with the frequencies: 4_seconds, minutely, 10_minutes, half_hourly, hourly, daily, weekly, monthly, quarterly and yearly
# consider multiple seasonalities for frequencies less than daily
SEASONALITY_VALS <- list()
SEASONALITY_VALS[[1]] <- c(21600, 151200, 7889400)
SEASONALITY_VALS[[2]] <- c(1440, 10080, 525960)
SEASONALITY_VALS[[3]] <- c(144, 1008, 52596)
SEASONALITY_VALS[[4]] <- c(48, 336, 17532)
SEASONALITY_VALS[[5]] <- c(24, 168, 8766)
SEASONALITY_VALS[[6]] <- 7
SEASONALITY_VALS[[7]] <- 365.25/7
SEASONALITY_VALS[[8]] <- 12 
SEASONALITY_VALS[[9]] <- 4
SEASONALITY_VALS[[10]] <- 1  

SEASONALITY_MAP <- list()

for(f in seq_along(FREQUENCIES)) {
    if (f > 10) next;
  SEASONALITY_MAP[[FREQUENCIES[f]]] <- SEASONALITY_VALS[[f]]
}
# Considered set of tsfeatures except mean, variance and stl_features
TSFEATURE_NAMES <- c( "max_kl_shift",
                      "max_level_shift",
                      "max_var_shift",
                      "acf_features",
                      "arch_stat",
                      "crossing_points",
                      "entropy",
                      "flat_spots",
                      "holt_parameters",
                      "hurst",
                      "lumpiness",
                      "nonlinearity",
                      "pacf_features",
                      "stability",
                      "unitroot_kpss",
                      "unitroot_pp"
)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



The R code for the function that calculates the features

In [ ]:
#@title R calculate_features
%%R

# This function calculates tsfeatures, catch22 features and BoxCox lambda values
# Parameters
# dataset_name - the name of the dataset
# input_file_name - name of the .tsf file corresponding with the dataset
# key - the name of the attribute that should be used as the key when creating the tsibble
# index - the name of the time attribute that should be used as the index when creating the tsibble
# feature_type  - tsfeatures, catch22 or lambda
calculate_features <- function(dataset_name, input_file_name, key = NULL, index = NULL, feature_type = "tsfeatures"){
  
  print(paste0("Started feature calculation: ", dataset_name))
  
  # Defining output file name
  if(feature_type == "lambda")
    output_file_name <- paste0(dataset_name, "_lambdas.csv")
  else
    output_file_name <- paste0(dataset_name, "_features.csv")
  
  
  # Loading data from the .tsf file
  loaded_data <- convert_tsf_to_tsibble(file.path(BASE_DIR, "tsf_data", input_file_name, fsep = "/"), VALUE_COL_NAME, key, index)
  dataset <- loaded_data[[1]]
  frequency <- loaded_data[[2]]
  
  if(!is.null(frequency))
    seasonality <- SEASONALITY_MAP[[frequency]]
  else
    seasonality <- 1
  
  all_serie_names <- unique(dataset$series_name)
  tslist <- list()
  
  for(s in seq_along(all_serie_names)){
    series_data <- dataset[dataset$series_name == as.character(all_serie_names[s]), ]
    
    if(is.null(index))
      series <- forecast:::msts(series_data[[VALUE_COL_NAME]], seasonal.periods = seasonality)
    else{
      start_date <- start(as.ts(series_data[, c(index, VALUE_COL_NAME)], frequency = max(seasonality)))
      
      if(length(start_date) == 1){ # Prepararing the start date according to the format required by stl_features such as peak and trough
        start_date <- c(floor(start_date), floor((start_date - floor(start_date)) * max(seasonality)))
      }
      
      series <- forecast:::msts(series_data[[VALUE_COL_NAME]], start = start_date, seasonal.periods = seasonality, ts.frequency = floor(max(seasonality)))
    }
    
    tslist[[s]] <- series
  }
  
  
  for(i in 1:length(tslist)){
    print(i)
    
    features <- NULL
    
    if(feature_type == "tsfeatures"){ # Calculating tsfeatures
      features <- tsfeatures:::tsfeatures(tslist[[i]], c("mean","var"), scale = FALSE, na.rm = TRUE)
      
      for(f in TSFEATURE_NAMES){
        calculated_features <- tsfeatures:::tsfeatures(tslist[[i]], features = f)
        
        if(sum(is.na(calculated_features)) > 0){ # if the calculated features contain missing values, then consider freequency as 1
          calculated_features <- tsfeatures:::tsfeatures(ts(tslist[[i]], frequency = 1), features = f)
          
          if(sum(is.na(calculated_features)) > 0){ # Still if there are missing values, modify the parameters of the corresponding function
            if(f == "max_kl_shift" | f == "max_level_shift" | f == "max_var_shift")
              calculated_features <- tsfeatures:::tsfeatures(tslist[[i]], features = f, width = 1)
            else{
              if(f == "arch_stat")
                calculated_features <- tsfeatures:::tsfeatures(tslist[[i]], features = f, lag = 1)
            }
          }
        }
        
        features <- bind_cols(features, calculated_features)
      }
      
      # Calculating stl_features
      tryCatch( 
        seasonal_features <- tsfeatures:::tsfeatures(tslist[[i]],"stl_features", s.window = 'periodic', robust = TRUE)
        , error = function(e) {
          tryCatch({
            seasonal_features <<- tsfeatures:::tsfeatures(tslist[[i]],"stl_features")
          }, error = function(e) {
            seasonal_features <<- tsfeatures:::tsfeatures(ts(tslist[[i]], frequency = 1),"stl_features") # Ignoring seasonality
          })
        })
      
      features <- bind_cols(features, seasonal_features)
      
      if(i == 1){
        all_features <- matrix(NA, nrow = length(tslist), ncol = ncol(features)) # Creating a matrix to store the calculated features
        colnames(all_features) <- colnames(features)
      }else{
        if(ncol(all_features) != ncol(features)){ # The features common to all series will be considered finally
          common_features <- intersect(colnames(all_features), colnames(features))
          all_features <- all_features[,common_features]
          features <- features[, common_features]
        }
      }
      
      all_features[i,] <- as.numeric(features)
      
    }else if(feature_type == "catch22"){ # Calculating catch22 features
      
      features <- catch22_all(tslist[[i]])
      
      if(i == 1){
        all_features <- matrix(NA, nrow = length(tslist), ncol = 22) # Creating a matrix to store the calculated features
        colnames(all_features) <- features$names
      }
      
      all_features[i,] <- features$values 
    }else{
      if(i == 1) # Calculating BoxCox lambda values
        lambdas <- forecast:::BoxCox.lambda(tslist[[i]])
      else
        lambdas <- c(lambdas, forecast:::BoxCox.lambda(tslist[[i]]))
    }
  }
  
  # Writing the calculated features into a file
  if(feature_type == "tsfeatures")
    write.table(all_features, file.path(BASE_DIR, "results", "tsfeatures", output_file_name, fsep = "/"), row.names = FALSE, col.names = TRUE, sep = ",", quote = FALSE)
  else if(feature_type == "catch22")
    write.table(all_features, file.path(BASE_DIR, "results", "catch22_features", output_file_name, fsep = "/"), row.names = FALSE, col.names = TRUE, sep = ",", quote = FALSE)
  else
    write.table(lambdas, file.path(BASE_DIR, "results", "lambdas", output_file_name, fsep = "/"), row.names = FALSE, col.names = FALSE, sep = ",", quote = FALSE)
}

We need to create the directory structure. **THIS SHOULD BE FIXED IN THE REPO**

In [ ]:
%%R
dir.create("TSForecasting/results/")
dir.create("TSForecasting/results/tsfeatures")
dir.create("TSForecasting/results/catch22_features")

In [ ]:
!ls -r TSForecasting/results

tsfeatures  catch22_features  catch22


In [ ]:
%%R

calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "catch22")

[1] "Started feature calculation: sample"
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(key)` instead of `key` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.


R[write to console]: Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 

## Part 2) Calculating features directly on Python

Load the function into python environment through rpy2 so
it can be later accessed directly from python code

In [ ]:
import rpy2.robjects as robjects
calculate_features = robjects.r['calculate_features']


we can now call `calculate_features` directly from python, here we have two examples using both the 'tsfeatures' and the 'catch22' features.
We will first remove the files just in case

In [ ]:
!mkdir TSForecasting/results
!mkdir TSForecasting/results/tsfeatures
!mkdir TSForecasting/results/catch22_features

mkdir: cannot create directory ‘TSForecasting/results’: File exists
mkdir: cannot create directory ‘TSForecasting/results/tsfeatures’: File exists
mkdir: cannot create directory ‘TSForecasting/results/catch22_features’: File exists


In [ ]:
!rm TSForecasting/results/tsfeatures/sample_features.csv
!rm TSForecasting/results/catch22_features/sample_features.csv

Compute the features, we capture the stdout output to remove the verbosiness  of the progress report

In [ ]:
%%capture cap --no-stderr
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "catch22")

The functions write the outputs to .csv files in the project directory
structure. We can quickly check if the experiments runs OK, we should see some features in the following files.

In [ ]:
!echo "head of the tsfeatures features"
!ls -r TSForecasting/results/tsfeatures
!head -5 TSForecasting/results/tsfeatures/sample_features.csv
!echo "head of the catch22 features"
!ls -r TSForecasting/results/catch22_features
!head -5 TSForecasting/results/catch22_features/sample_features.csv

head of the tsfeatures features
sample_features.csv
mean,var,max_kl_shift,time_kl_shift,max_level_shift,time_level_shift,max_var_shift,time_var_shift,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,flat_spots,alpha,beta,hurst,lumpiness,nonlinearity,x_pacf5,diff1x_pacf5,diff2x_pacf5,seas_pacf,stability,unitroot_kpss,unitroot_pp,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough
105.652455128205,669.530932765715,7.83133691877834,12,1.86816865937618,89,2.15074891552011,28,0.710805797539225,1.51175743379058,-0.239716429411997,0.105249143633781,-0.539721534779089,0.372057773793558,-0.0145394944907041,0.257725362872141,22,0.795329846811296,14,0.628482857552452,0.000100053599224661,0.989518146632079,0.152842055714045,0.315470866286382,0.530751963124808,0.104820622256713,0.656220390192105,-0.0296746509672276,0.627384823226218,0.459213533792908,-39.7197694054302,1,12,0.572759852670852,0.000